In [1]:
import os
os.environ["SERPER_API_KEY"] = ""  # serper.dev API key
os.environ["OPENAI_API_KEY"] = "" # OpenAI API key

In [2]:
import uuid
import functools
import requests
from langchain.callbacks.manager import Callbacks
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.tools import tool

In [3]:
from crewai import Crew, Process
from crewai import Task

In [4]:
from crewai import Agent

In [10]:
import coffifilter

In [11]:
coffifilter.init(
    redis_host="",
    redis_port=11552,
    redis_db=0,
    redis_password=""
)

In [12]:
@tool
@coffifilter.coffi_filter("summarize_tool")
def summarize_tool(url: str, callbacks: Callbacks = None):
    """Summarize a website."""
    text = requests.get(url).text
    summary_chain = (
        ChatPromptTemplate.from_template(
            "Summarize the following text:\n<TEXT {uid}>\n" "{text}" "\n</TEXT {uid}>"
        ).partial(uid=lambda: uuid.uuid4())
        | ChatOpenAI(model="gpt-4o")
        | StrOutputParser()
    ).with_config(run_name="Summarize Text")
    return summary_chain.invoke(
        {"text": text},
        {"callbacks": callbacks},
    )

In [14]:
# Creating a summarizer agent that can summarize a webpage
summarizer = Agent(
  role='Summarizer',
  goal='Summarize the content from a webpage: {url}. If you cannot use your tools, return a graceful response and exit.',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to distill and share knowledge that could change"
    "the world."
  ),
  tools=[summarize_tool],
  allow_delegation=True
)

In [16]:
# Summarize task
summarize_task = Task(
  description=(
    "Summarize the content from a webpage: {url}."
    "Your final report should clearly articulate the key points."
  ),
  expected_output='A comprehensive summary.',
  tools=[summarize_tool],
  agent=summarizer,
)

In [17]:
# Forming the tech-focused crew with some enhanced configurations
summ_crew = Crew(
  agents=[summarizer],
  tasks=[summarize_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [22]:
# Starting the task execution process with enhanced feedback
summ_result = summ_crew.kickoff(inputs={'url': 'https://facebook.com'})
print(summ_result)



> Entering new CrewAgentExecutor chain...
First, I need to use the summarize_tool to gather information from the provided URL. 

Action: 
summarize_tool

Action Input: 
{"url": "https://facebook.com"}
 

Tool 'summarize_tool' is not enabled

Thought: 
The tool I planned to use, summarize_tool, is not enabled. I can't proceed without it. 

Final Answer: 
I'm sorry, but I'm unable to provide a summary for the given webpage at this moment as the required tool for the task is not enabled.

> Finished chain.
I'm sorry, but I'm unable to provide a summary for the given webpage at this moment as the required tool for the task is not enabled.
